In [45]:
import numpy as np
import pandas as pd
import numpy as n
from feature_engine.encoding import OrdinalEncoder
from sklearn.neighbors import KNeighborsClassifier as KNN
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.metrics import roc_curve, auc

In [46]:
data = pd.read_pickle('data/aus_weather_cln.pkl')
data

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,...,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow
0,2008-12-01,Albury,13.4,22.900000,0.6,1.4,9.5,0.073518,44.0,0.062725,...,71.0,22.0,1007.7,1007.1,8.0,4.0,16.9,21.8,No,No
1,2008-12-02,Albury,7.4,25.100000,0.0,0.8,0.3,0.060821,44.0,0.059164,...,44.0,25.0,1010.6,1007.8,7.0,0.0,17.2,24.3,No,No
2,2008-12-03,Albury,12.9,25.700000,0.0,6.2,4.7,0.067063,46.0,0.062725,...,38.0,30.0,1007.6,1008.7,0.0,2.0,21.0,23.2,No,No
3,2008-12-04,Albury,9.2,28.000000,0.0,3.8,3.0,0.052729,24.0,0.068892,...,45.0,16.0,1017.6,1012.8,4.0,3.0,18.1,26.5,No,No
4,2008-12-05,Albury,17.5,32.300000,1.0,3.6,11.6,0.073518,41.0,0.058243,...,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,Uluru,2.8,23.400000,0.0,5.0,10.8,0.068074,31.0,0.068892,...,51.0,24.0,1024.6,1020.3,8.0,7.0,10.1,22.4,No,No
145456,2017-06-22,Uluru,3.6,25.300000,0.0,5.4,5.6,0.048900,22.0,0.068892,...,56.0,21.0,1023.5,1019.1,4.0,7.0,10.9,24.5,No,No
145457,2017-06-23,Uluru,5.4,26.900000,0.0,3.2,0.0,0.068747,37.0,0.068892,...,53.0,24.0,1021.0,1016.8,4.0,0.0,12.5,26.1,No,No
145458,2017-06-24,Uluru,7.8,27.000000,0.0,1.6,6.6,0.069847,28.0,0.067737,...,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,No,No


In [47]:
data['Date'] = pd.to_datetime(data['Date'])

In [48]:
data['RainToday'] = data['RainToday'].apply(lambda x: 1 if x == 'Yes' else 0)
data['RainTomorrow'] = data['RainTomorrow'].apply(lambda x: 1 if x == 'Yes' else 0)
data[['RainToday', 'RainTomorrow']]

,RainToday,RainTomorrow
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
145455,0,0
145456,0,0
145457,0,0
145458,0,0


In [49]:
data['Year'] = data['Date'].dt.year
data['Month'] = data['Date'].dt.month
data['Quarter'] = data['Date'].dt.quarter
data.drop('Date', axis=1, inplace=True)
data

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Quarter
0,Albury,13.4,22.900000,0.6,1.4,9.5,0.073518,44.0,0.062725,0.062739,...,1007.1,8.0,4.0,16.9,21.8,0,0,2008,12,4
1,Albury,7.4,25.100000,0.0,0.8,0.3,0.060821,44.0,0.059164,0.067469,...,1007.8,7.0,0.0,17.2,24.3,0,0,2008,12,4
2,Albury,12.9,25.700000,0.0,6.2,4.7,0.067063,46.0,0.062725,0.067469,...,1008.7,0.0,2.0,21.0,23.2,0,0,2008,12,4
3,Albury,9.2,28.000000,0.0,3.8,3.0,0.052729,24.0,0.068892,0.059989,...,1012.8,4.0,3.0,18.1,26.5,0,0,2008,12,4
4,Albury,17.5,32.300000,1.0,3.6,11.6,0.073518,41.0,0.058243,0.060876,...,1006.0,7.0,8.0,17.8,29.7,0,0,2008,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,Uluru,2.8,23.400000,0.0,5.0,10.8,0.068074,31.0,0.068892,0.055685,...,1020.3,8.0,7.0,10.1,22.4,0,0,2017,6,2
145456,Uluru,3.6,25.300000,0.0,5.4,5.6,0.048900,22.0,0.068892,0.062849,...,1019.1,4.0,7.0,10.9,24.5,0,0,2017,6,2
145457,Uluru,5.4,26.900000,0.0,3.2,0.0,0.068747,37.0,0.068892,0.062739,...,1016.8,4.0,0.0,12.5,26.1,0,0,2017,6,2
145458,Uluru,7.8,27.000000,0.0,1.6,6.6,0.069847,28.0,0.067737,0.062849,...,1016.5,3.0,2.0,15.1,26.0,0,0,2017,6,2


In [50]:
location_encoder = OrdinalEncoder(encoding_method='arbitrary',
                                  variables=['Location'])
location_encoder.fit(data)
# location_encoder.encoder_dict_
data = location_encoder.transform(data)
data

,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,...,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Year,Month,Quarter
0,0,13.4,22.900000,0.6,1.4,9.5,0.073518,44.0,0.062725,0.062739,...,1007.1,8.0,4.0,16.9,21.8,0,0,2008,12,4
1,0,7.4,25.100000,0.0,0.8,0.3,0.060821,44.0,0.059164,0.067469,...,1007.8,7.0,0.0,17.2,24.3,0,0,2008,12,4
2,0,12.9,25.700000,0.0,6.2,4.7,0.067063,46.0,0.062725,0.067469,...,1008.7,0.0,2.0,21.0,23.2,0,0,2008,12,4
3,0,9.2,28.000000,0.0,3.8,3.0,0.052729,24.0,0.068892,0.059989,...,1012.8,4.0,3.0,18.1,26.5,0,0,2008,12,4
4,0,17.5,32.300000,1.0,3.6,11.6,0.073518,41.0,0.058243,0.060876,...,1006.0,7.0,8.0,17.8,29.7,0,0,2008,12,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,48,2.8,23.400000,0.0,5.0,10.8,0.068074,31.0,0.068892,0.055685,...,1020.3,8.0,7.0,10.1,22.4,0,0,2017,6,2
145456,48,3.6,25.300000,0.0,5.4,5.6,0.048900,22.0,0.068892,0.062849,...,1019.1,4.0,7.0,10.9,24.5,0,0,2017,6,2
145457,48,5.4,26.900000,0.0,3.2,0.0,0.068747,37.0,0.068892,0.062739,...,1016.8,4.0,0.0,12.5,26.1,0,0,2017,6,2
145458,48,7.8,27.000000,0.0,1.6,6.6,0.069847,28.0,0.067737,0.062849,...,1016.5,3.0,2.0,15.1,26.0,0,0,2017,6,2


In [77]:
location_encoder.encoder_dict_

{'Location': {'Albury': 0,
  'BadgerysCreek': 1,
  'Cobar': 2,
  'CoffsHarbour': 3,
  'Moree': 4,
  'Newcastle': 5,
  'NorahHead': 6,
  'NorfolkIsland': 7,
  'Penrith': 8,
  'Richmond': 9,
  'Sydney': 10,
  'SydneyAirport': 11,
  'WaggaWagga': 12,
  'Williamtown': 13,
  'Wollongong': 14,
  'Canberra': 15,
  'Tuggeranong': 16,
  'MountGinini': 17,
  'Ballarat': 18,
  'Bendigo': 19,
  'Sale': 20,
  'MelbourneAirport': 21,
  'Melbourne': 22,
  'Mildura': 23,
  'Nhil': 24,
  'Portland': 25,
  'Watsonia': 26,
  'Dartmoor': 27,
  'Brisbane': 28,
  'Cairns': 29,
  'GoldCoast': 30,
  'Townsville': 31,
  'Adelaide': 32,
  'MountGambier': 33,
  'Nuriootpa': 34,
  'Woomera': 35,
  'Albany': 36,
  'Witchcliffe': 37,
  'PearceRAAF': 38,
  'PerthAirport': 39,
  'Perth': 40,
  'SalmonGums': 41,
  'Walpole': 42,
  'Hobart': 43,
  'Launceston': 44,
  'AliceSprings': 45,
  'Darwin': 46,
  'Katherine': 47,
  'Uluru': 48}}

### regression feature selection

In [51]:
temp = data.drop('RainTomorrow', axis=1)
y = data['RainTomorrow']
features = SelectKBest(score_func=f_regression, k=10)
selected_features = features.fit_transform(temp, y)
selected_features

array([[ 0.6,  9.5, 44. , ...,  8. ,  4. ,  0. ],
       [ 0. ,  0.3, 44. , ...,  7. ,  0. ,  0. ],
       [ 0. ,  4.7, 46. , ...,  0. ,  2. ,  0. ],
       ...,
       [ 0. ,  0. , 37. , ...,  4. ,  0. ,  0. ],
       [ 0. ,  6.6, 28. , ...,  3. ,  2. ,  0. ],
       [ 0. , 10.9, 54. , ...,  8. ,  8. ,  0. ]])

In [52]:
input_feature = ['Location', 'Rainfall', 'Sunshine', 'WindGustSpeed', 'Humidity9am', 'Humidity3pm',
                 'Pressure9am', 'Pressure3pm', 'Cloud9am', 'Cloud3pm', 'RainToday']
train_x, test_x, train_y, test_y = train_test_split(temp[input_feature], y, test_size=0.2, random_state=64)
train_x.shape, test_x.shape

((116368, 11), (29092, 11))

In [53]:
with open('model/rain_logistic_model.pkl', 'rb') as file:
    logistic_model = pd.read_pickle(file)

with open('model/rain_knn_model.pkl', 'rb') as file:
    knn_model = pd.read_pickle(file)

with open('model/rain_tree_model.pkl', 'rb') as file:
    tree_model = pd.read_pickle(file)

In [54]:
lp = logistic_model.predict_proba(test_x)[:, 1]
roc_curve(test_y, lp)

(array([0.        , 0.        , 0.        , ..., 0.99605426, 0.99605426,
        1.        ]),
 array([0.00000000e+00, 1.52998776e-04, 2.90697674e-03, ...,
        9.99847001e-01, 1.00000000e+00, 1.00000000e+00]),
 array([1.99740932, 0.99740932, 0.98717314, ..., 0.00595758, 0.0059552 ,
        0.0028481 ]))

In [55]:
kp = knn_model.predict_proba(test_x)[:, 1]
fpr, tpr, thresholds = roc_curve(test_y, kp)

In [56]:
tp = tree_model.predict_proba(test_x)[:, 1]
roc_curve(test_y, tp)

(array([0.        , 0.00230537, 0.00261571, 0.00687179, 0.0078028 ,
        0.01064018, 0.0117042 , 0.01578294, 0.01870899, 0.02633446,
        0.03076787, 0.03635396, 0.04380209, 0.05160489, 0.05878702,
        0.06401844, 0.07355027, 0.08698351, 0.10928356, 0.12475616,
        0.13898741, 0.16700656, 0.17245966, 0.21998581, 0.24272921,
        0.26746764, 0.35081575, 0.44773009, 0.47326654, 0.50691612,
        0.62515517, 0.93305551, 1.        ]),
 array([0.        , 0.07986536, 0.09761322, 0.17258262, 0.19186047,
        0.22154223, 0.23087515, 0.26132191, 0.28044676, 0.31716646,
        0.34195226, 0.36735006, 0.40559976, 0.42350061, 0.44430845,
        0.45945532, 0.4869951 , 0.52019584, 0.56777846, 0.59669523,
        0.62423501, 0.65636475, 0.66126071, 0.70807834, 0.72796818,
        0.74847001, 0.82114443, 0.875153  , 0.8875459 , 0.90100979,
        0.94706242, 0.98944308, 1.        ]),
 array([1.92300885, 0.92300885, 0.8875969 , 0.8384755 , 0.8304    ,
        0.77843302, 0.70

In [57]:
pred = logistic_model.predict(test_x)
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [69]:
comp_df = pd.DataFrame(np.array(test_y), columns=['true label'])
comp_df['predicted_label'] = list(pred)

,true label,predicted_label
0,0,0
1,0,0
2,0,0
3,0,0
4,0,0
...,...,...
29087,0,0
29088,0,0
29089,0,0
29090,0,0


In [70]:
true_labels = comp_df[comp_df['true label'] == 1]
false_labels = comp_df[comp_df['true label'] == 0]

((6536, 2), (22556, 2))

In [75]:
tp = np.sum(true_labels['true label'] == true_labels['predicted_label'])
tn = true_labels.shape[0] - tp
fp = np.sum(false_labels['true label'] == false_labels['predicted_label'])
fn = false_labels.shape[0] - fp

tp, tn, fp, fn

(6536, 0, 21361, 1195)

In [81]:
logistic_model.predict_proba(np.asarray([20, 6.2, 10.5, 98.0, 48.0, 32.0, 1002.900000, 999.300000, 2.0, 3.0, 1]).reshape(1, -1))[0][1]

0.5156439490362514